In [5]:
from pathlib import Path
output_path = 'S:/test/tjos/tests'
output_dir = Path(output_path).parent
output_dir.mkdir(parents=True, exist_ok=True)

In [8]:
from sleap import *
from behavior_utils import *
from sleap.info import write_tracking_h5

In [6]:
root_path = 'S:'
experiment = 'clickbait-motivate'
events_dir = 'sleap'
session_type = 'm'

# Get paths to relevant files
slp_paths = get_file_paths(f"{root_path}/{experiment}/sleap", 'slp', '', session_type=session_type, print_paths=True, print_n=np.inf)

Found 25 .slp files
0 S:\clickbait-motivate\sleap\7001\m1\7001_m1_sleap.h5.slp
1 S:\clickbait-motivate\sleap\7001\m2\7001_m2_sleap.h5.slp
2 S:\clickbait-motivate\sleap\7001\m3\7001_m3_sleap.h5.slp
3 S:\clickbait-motivate\sleap\7001\m4\7001_m4_sleap.h5.slp
4 S:\clickbait-motivate\sleap\7001\m5\7001_m5_sleap.h5.slp
5 S:\clickbait-motivate\sleap\7001\m6\7001_m6_sleap.h5.slp
6 S:\clickbait-motivate\sleap\7002\m0\7002_m0_sleap.h5.slp
7 S:\clickbait-motivate\sleap\7002\m1\7002_m1_sleap.h5.slp
8 S:\clickbait-motivate\sleap\7002\m2\7002_m2_sleap.h5.slp
9 S:\clickbait-motivate\sleap\7002\m3\7002_m3_sleap.h5.slp
10 S:\clickbait-motivate\sleap\7002\m4\7002_m4_sleap.h5.slp
11 S:\clickbait-motivate\sleap\7003\m0\7003_m0_sleap.h5.slp
12 S:\clickbait-motivate\sleap\7003\m1\7003_m1_sleap.h5.slp
13 S:\clickbait-motivate\sleap\7003\m2\7003_m2_sleap.h5.slp
14 S:\clickbait-motivate\sleap\7003\m3\7003_m3_sleap.h5.slp
15 S:\clickbait-motivate\sleap\7003\m4\7003_m4_sleap.h5.slp
16 S:\clickbait-motivate\sleap

In [33]:
for path in slp_paths:
    mouse_id = path.parts[-3]
    session_id = path.parts[-2]
    slp = sleap.load_file(f"{path}")
    slp.export_csv(f"{path.parent}/{mouse_id}_{session_id}_sleap.csv")
    print(f"Processed SLEAP .csv for {mouse_id}-{session_id}")

Processed SLEAP .csv for 7001-m1
Processed SLEAP .csv for 7001-m2
Processed SLEAP .csv for 7001-m3
Processed SLEAP .csv for 7001-m4
Processed SLEAP .csv for 7001-m5
Processed SLEAP .csv for 7001-m6
Processed SLEAP .csv for 7002-m0
Processed SLEAP .csv for 7002-m1
Processed SLEAP .csv for 7002-m2
Processed SLEAP .csv for 7002-m3
Processed SLEAP .csv for 7002-m4
Processed SLEAP .csv for 7003-m0
Processed SLEAP .csv for 7003-m1
Processed SLEAP .csv for 7003-m2
Processed SLEAP .csv for 7003-m3
Processed SLEAP .csv for 7003-m4
Processed SLEAP .csv for 7003-m5
Processed SLEAP .csv for 7003-m6
Processed SLEAP .csv for 7004-m0
Processed SLEAP .csv for 7004-m1
Processed SLEAP .csv for 7004-m2
Processed SLEAP .csv for 7004-m3
Processed SLEAP .csv for 7004-m4
Processed SLEAP .csv for 7004-m5
Processed SLEAP .csv for 7004-m6


In [36]:
# Check paths
sleap_paths = get_file_paths(f"{root_path}/{experiment}/sleap", 'csv', 'sleap', session_type=session_type, print_paths=True, print_n=np.inf)

Found 25 sleap.csv files
0 S:\clickbait-motivate\sleap\7001\m1\m1_7001_sleap.csv
1 S:\clickbait-motivate\sleap\7001\m2\m2_7001_sleap.csv
2 S:\clickbait-motivate\sleap\7001\m3\m3_7001_sleap.csv
3 S:\clickbait-motivate\sleap\7001\m4\m4_7001_sleap.csv
4 S:\clickbait-motivate\sleap\7001\m5\m5_7001_sleap.csv
5 S:\clickbait-motivate\sleap\7001\m6\m6_7001_sleap.csv
6 S:\clickbait-motivate\sleap\7002\m0\m0_7002_sleap.csv
7 S:\clickbait-motivate\sleap\7002\m1\m1_7002_sleap.csv
8 S:\clickbait-motivate\sleap\7002\m2\m2_7002_sleap.csv
9 S:\clickbait-motivate\sleap\7002\m3\m3_7002_sleap.csv
10 S:\clickbait-motivate\sleap\7002\m4\m4_7002_sleap.csv
11 S:\clickbait-motivate\sleap\7003\m0\m0_7003_sleap.csv
12 S:\clickbait-motivate\sleap\7003\m1\m1_7003_sleap.csv
13 S:\clickbait-motivate\sleap\7003\m2\m2_7003_sleap.csv
14 S:\clickbait-motivate\sleap\7003\m3\m3_7003_sleap.csv
15 S:\clickbait-motivate\sleap\7003\m4\m4_7003_sleap.csv
16 S:\clickbait-motivate\sleap\7003\m5\m5_7003_sleap.csv
17 S:\clickbait-